In [1]:
!pip install optuna

# 簡単なハイパーチューニング

In [2]:
import optuna
import tensorflow as tf

## 基本的な使用方法
1. まずobjective関数を定義
    - 第一引数はtrial（ここにoptunaの機能群が渡される）
    - returnは最適化の基準に用いる値（AccやLossなど） 
2. create_studyで最適化の初期設定を決める（最大化か最小化か等の設定）
3. optimize(objective, n_trials=試行回数)という形でチューニング
4. 最適なパラメータの値やその時のスコアがbest_params, best_valueに保存される

In [3]:
# 第一引数"trial"にoptunaの機能群が渡されるのでtrial.XXXという形で使用できる
def objective(trial):
    # チューニングに用いる候補を変数xに格納
    x = trial.suggest_categorical("node_num", [-3, -2, -1, 0, 1, 2])
    result = x**2
    return result


In [4]:
# 最適化する方向を決める（最小化か最大化か）今回は最大化をしてみる
study = optuna.create_study(direction='maximize')
# optimizeメソッドの第一引数に作成した関数を、n_trialsに探索回数を指定
study.optimize(objective, n_trials=10)


[I 2021-06-17 02:34:52,405] A new study created in memory with name: no-name-c703484a-72c6-4cbb-b853-6a9ab21a8b5f
[I 2021-06-17 02:34:52,410] Trial 0 finished with value: 9.0 and parameters: {'node_num': -3}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,413] Trial 1 finished with value: 9.0 and parameters: {'node_num': -3}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,416] Trial 2 finished with value: 9.0 and parameters: {'node_num': -3}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,419] Trial 3 finished with value: 1.0 and parameters: {'node_num': -1}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,422] Trial 4 finished with value: 4.0 and parameters: {'node_num': 2}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,425] Trial 5 finished with value: 0.0 and parameters: {'node_num': 0}. Best is trial 0 with value: 9.0.
[I 2021-06-17 02:34:52,428] Trial 6 finished with value: 1.0 and parameters: {'node_num': 1}. Best is trial 0 with value

In [5]:
# best_paramsが最適になった際のパラメータの値
print(study.best_params)
# best_valueがその時の実際のスコア
print(study.best_value)
# 当然 x^2 を求めているので (-3)^2 が最もよい値となる


{'node_num': -3}
9.0


# CIFARを用いたチューニング例

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D


In [7]:
# データ読み込みと正規化、OneHot化等の前処理
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


In [8]:
def objective(trial):
    # 念のためリフレッシュ
    tf.keras.backend.clear_session()
    # suggest_intでは100～500のintを選択してくれる
    num_fc_units = trial.suggest_int('num_fc_units', 100, 500)
    # チューニングするモデルを作成
    model = keras.Sequential()
    model.add(
        Conv2D(32, (3, 3),
               padding='same',
               input_shape=(32, 32, 3),
               activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    # 今回は全結合部分のノード数をチューニング
    model.add(Dense(num_fc_units, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=2**12, epochs=10, verbose=0)
    # model.evaluateは[loss, acc]が返されるのでaccのみを抽出
    result = model.evaluate(x_test, y_test, verbose=0)[1]
    return result


In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)


[I 2021-06-17 02:34:53,503] A new study created in memory with name: no-name-eeff3721-c819-4e3a-b42d-386775a8e587
[I 2021-06-17 02:35:16,569] Trial 0 finished with value: 0.5202999711036682 and parameters: {'num_fc_units': 196}. Best is trial 0 with value: 0.5202999711036682.
[I 2021-06-17 02:35:30,287] Trial 1 finished with value: 0.5442000031471252 and parameters: {'num_fc_units': 451}. Best is trial 1 with value: 0.5442000031471252.
[I 2021-06-17 02:35:52,318] Trial 2 finished with value: 0.5669000148773193 and parameters: {'num_fc_units': 194}. Best is trial 2 with value: 0.5669000148773193.
[I 2021-06-17 02:36:15,045] Trial 3 finished with value: 0.532800018787384 and parameters: {'num_fc_units': 281}. Best is trial 2 with value: 0.5669000148773193.
[I 2021-06-17 02:36:37,062] Trial 4 finished with value: 0.5548999905586243 and parameters: {'num_fc_units': 193}. Best is trial 2 with value: 0.5669000148773193.


In [10]:
study.best_params, study.best_value


({'num_fc_units': 194}, 0.5669000148773193)

# 課題で用いたコード
※データの読み込み等の部分は割愛しているのでこのままでは動かないです

In [ ]:
# コードが長くなるのでモデルの定義部分を別関数としている
# 引数num_unitで指定した数のノード数のRNNを構築し、returnでそのモデルを返す
def create_model_trial(num_unit=128):
    model = tf.keras.Sequential()
    model.add(
        tf.keras.layers.SimpleRNN(num_unit,
                                  input_shape=x_train[0].shape,
                                  activation="tanh",
                                  return_sequences=True,
                                  kernel_initializer="random_normal",
                                  recurrent_initializer="orthogonal"))
    model.add(
        tf.keras.layers.Dense(y_train[0].shape[1],
                              use_bias=True,
                              activation="softmax"))

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics="categorical_accuracy")
    return model


def objective(trial):
    tf.keras.backend.clear_session()

    # Denseノード数をnum_unitsとしてチューニング
    num_units = trial.suggest_categorical("num_unit",
                                          [2**i
                                           for i in range(5, 11)])  # 32-1024

    # チューニングするモデルを作成
    model = create_model_trial(num_units)

    # モデルのパラメータだけでなく、以下のようにバッチサイズやエポック数もチューニング可能
    history = model.fit(
        x_train,
        y_train,
        batch_size=trial.suggest_categorical("num_batch",
                                             [2**i
                                              for i in range(14)]),  # 1-2048
        epochs=trial.suggest_categorical("num_epoch",
                                         [2**i
                                          for i in range(5, 10)]),  # 16-512
        verbose=0)

    result = model.evaluate(x=x_test, y=y_test)[1]
    return result


In [ ]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1)


In [ ]:
# study.best_params, study.best_value
